<a href="https://colab.research.google.com/github/kgndirangu/project-4/blob/main/machine_learning_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,073 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,927 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,514 kB]
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:13 http://security.ubuntu.com/u

In [ ]:
# Import Dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
from pathlib import Path
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
from pyspark import SparkFiles
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

In [ ]:
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# Read in data from S3 Buckets

url = "https://project4-ny-health-data-1.s3.amazonaws.com/inpatient_hospital_2.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("inpatient_hospital_2.csv"), header=True, inferSchema=True, quote="\"", escape="\"")

# Show Spark DataFrame
df.show()

+-------------------+---------------+-----------+--------------------+---------+-------------------+------+-----+-----------------+--------------+-----------------+-------------------+------------------+-------------------------+------------------+-------------------------+------------+--------------------+------------+--------------------+----------------------------+-----------------------------------+---------------------+--------------------------------+-------------------+------------------------------+-------------+-----------+--------+-------------+--------+-------------+---------+-------------+-------+-------------+---------+-------------+--------+-------------+---------+--------------+---------+-------------+---------+-------------+
|Health Service Area|Hospital County|Facility ID|       Facility Name|Age Group|Zip Code - 3 digits|Gender| Race|        Ethnicity|Length of Stay|Type of Admission|Patient Disposition|CCS Diagnosis Code|CCS Diagnosis Description|CCS Procedure Code|

In [ ]:
df.dtypes

[('Health Service Area', 'string'),
 ('Hospital County', 'string'),
 ('Facility ID', 'double'),
 ('Facility Name', 'string'),
 ('Age Group', 'string'),
 ('Zip Code - 3 digits', 'string'),
 ('Gender', 'string'),
 ('Race', 'string'),
 ('Ethnicity', 'string'),
 ('Length of Stay', 'string'),
 ('Type of Admission', 'string'),
 ('Patient Disposition', 'string'),
 ('CCS Diagnosis Code', 'double'),
 ('CCS Diagnosis Description', 'string'),
 ('CCS Procedure Code', 'double'),
 ('CCS Procedure Description', 'string'),
 ('APR DRG Code', 'int'),
 ('APR DRG Description', 'string'),
 ('APR MDC Code', 'int'),
 ('APR MDC Description', 'string'),
 ('APR Severity of Illness Code', 'int'),
 ('APR Severity of Illness Description', 'string'),
 ('APR Risk of Mortality', 'string'),
 ('APR Medical Surgical Description', 'string'),
 ('Source of Payment 1', 'string'),
 ('Emergency Department Indicator', 'string'),
 ('Total Charges', 'double'),
 ('Total Costs', 'double'),
 ('AgeIndex', 'double'),
 ('catVecAge', '

In [ ]:
clean_df = df.drop("Health Service Area",	"Hospital County", "Facility Name",	"Age Group", "Gender", "Race",	"Ethnicity", "Type of Admission",	"Patient Disposition", "CCS Diagnosis Description", "CCS Procedure Description","APR DRG Description", "APR MDC Description", 'APR Severity of Illness Description', 'APR Risk of Mortality', 'APR Medical Surgical Description','Source of Payment 1',"Zip Code - 3 digits", 'Emergency Department Indicator')

In [ ]:
clean_df.head()

Row(Facility ID=1437.0, Length of Stay='2', CCS Diagnosis Code=218.0, CCS Procedure Code=228.0, APR DRG Code=640, APR MDC Code=15, APR Severity of Illness Code=1, Total Charges=2468.95, Total Costs=1423.82, AgeIndex=3.0, catVecAge='(4,[3],[1.0])', SexIndex=1.0, catVecSex='(2,[1],[1.0])', RaceIndex=0.0, catVecRace='(3,[0],[1.0])', ERIndex=0.0, catVecER='(1,[0],[1.0])', Pay1Index=2.0, catVecPay1='(9,[2],[1.0])', AdmIndex=2.0, catVecAdm='(5,[2],[1.0])', DispIndex=0.0, catVecDisp='(17,[0],[1.0])', MortIndex=0.0, catVecMort='(3,[0],[1.0])', SurgIndex=0.0, catVecSurg='(1,[0],[1.0])')

In [ ]:
clean_df.withColumn("Length of Stay",col("Length of Stay").cast("int"))

DataFrame[Facility ID: double, Length of Stay: int, CCS Diagnosis Code: double, CCS Procedure Code: double, APR DRG Code: int, APR MDC Code: int, APR Severity of Illness Code: int, Total Charges: double, Total Costs: double, AgeIndex: double, catVecAge: string, SexIndex: double, catVecSex: string, RaceIndex: double, catVecRace: string, ERIndex: double, catVecER: string, Pay1Index: double, catVecPay1: string, AdmIndex: double, catVecAdm: string, DispIndex: double, catVecDisp: string, MortIndex: double, catVecMort: string, SurgIndex: double, catVecSurg: string]

In [ ]:
clean_df2 = clean_df.drop("catVecAge", "catVecSex", "catVecRace", "catVecER", "catVecPay1", "catVecAdm","catVecDisp","catVecMort", "catVecSurg")

In [ ]:
pandas_df = clean_df2.toPandas()

In [ ]:
pandas_df

,Facility ID,Length of Stay,CCS Diagnosis Code,CCS Procedure Code,APR DRG Code,APR MDC Code,APR Severity of Illness Code,Total Charges,Total Costs,AgeIndex,SexIndex,RaceIndex,ERIndex,Pay1Index,AdmIndex,DispIndex,MortIndex,SurgIndex
0,1437.0,2,218.0,228.0,640,15,1,2468.95,1423.82,3.0,1.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0
1,1437.0,3,218.0,228.0,640,15,1,6647.18,3335.77,3.0,1.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0
2,1437.0,2,218.0,228.0,640,15,1,5702.18,2712.70,3.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
3,1437.0,2,218.0,228.0,640,15,1,2125.58,1337.46,3.0,1.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0
4,1437.0,3,218.0,220.0,640,15,1,3050.58,1952.85,3.0,1.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443897,3975.0,5,245.0,203.0,204,5,2,36663.34,12249.99,2.0,0.0,2.0,0.0,1.0,0.0,2.0,1.0,0.0
443898,3975.0,25,99.0,177.0,468,11,2,156389.24,59015.68,2.0,0.0,2.0,0.0,1.0,0.0,2.0,1.0,0.0
443899,3975.0,6,14.0,78.0,221,6,2,53022.38,19448.05,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
443900,3975.0,3,96.0,193.0,200,5,3,18968.74,6874.04,2.0,0.0,1.0,0.0,4.0,0.0,0.0,2.0,0.0


In [ ]:
pandas_df["Length of Stay"] = pandas_df["Length of Stay"].replace("120 +", 121)

In [ ]:
pandas_df["Length of Stay"].unique()

array(['2', '3', '5', '4', '19', '1', '10', '13', '37', '27', '34', '26',
       '6', '32', '12', '14', '18', '7', '20', '51', '8', '80', '11', '9',
       '57', 121, '25', '29', '24', '33', '17', '23', '21', '15', '30',
       '28', '84', '36', '40', '16', '52', '102', '41', '64', '22', '65',
       '31', '55', '38', '46', '45', '58', '35', '69', '59', '49', '95',
       '43', '47', '61', '50', '42', '48', '94', '71', '53', '97', '75',
       '91', '86', '67', '60', '68', '78', '106', '63', '44', '39', '62',
       '83', '117', '90', '101', '99', '96', '112', '70', '79', '109',
       '77', '56', '103', '98', '54', '85', '89', '72', '73', '74', '92',
       '113', '81', '88', '87', '93', '76', '104', '115', '107', '118',
       '66', '114', '100', '105', '108', '116', '110', '82', '111', '119'],
      dtype=object)

In [ ]:
pandas_df["Length of Stay"] = pandas_df["Length of Stay"].astype(int)

In [ ]:
pandas_df.dtypes

Facility ID                     float64
Length of Stay                    int64
CCS Diagnosis Code              float64
CCS Procedure Code              float64
APR DRG Code                      int32
APR MDC Code                      int32
APR Severity of Illness Code      int32
Total Charges                   float64
Total Costs                     float64
AgeIndex                        float64
SexIndex                        float64
RaceIndex                       float64
ERIndex                         float64
Pay1Index                       float64
AdmIndex                        float64
DispIndex                       float64
MortIndex                       float64
SurgIndex                       float64
dtype: object

In [ ]:
# Set X and y
X=pandas_df.drop("Total Charges", axis =1).values
y= pandas_df["Total Charges"].values

In [ ]:
X[:5]

array([[1.43700e+03, 2.00000e+00, 2.18000e+02, 2.28000e+02, 6.40000e+02,
        1.50000e+01, 1.00000e+00, 1.42382e+03, 3.00000e+00, 1.00000e+00,
        0.00000e+00, 0.00000e+00, 2.00000e+00, 2.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.43700e+03, 3.00000e+00, 2.18000e+02, 2.28000e+02, 6.40000e+02,
        1.50000e+01, 1.00000e+00, 3.33577e+03, 3.00000e+00, 1.00000e+00,
        0.00000e+00, 0.00000e+00, 2.00000e+00, 2.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.43700e+03, 2.00000e+00, 2.18000e+02, 2.28000e+02, 6.40000e+02,
        1.50000e+01, 1.00000e+00, 2.71270e+03, 3.00000e+00, 1.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 2.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.43700e+03, 2.00000e+00, 2.18000e+02, 2.28000e+02, 6.40000e+02,
        1.50000e+01, 1.00000e+00, 1.33746e+03, 3.00000e+00, 1.00000e+00,
        0.00000e+00, 0.00000e+00, 2.00000e+00, 2.00000e+00, 0.00000e+00,
        0.00000e+00

In [ ]:
y[:5]

array([2468.95, 6647.18, 5702.18, 2125.58, 3050.58])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=42)

In [ ]:
# Preprocess numerical data for neural network

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  Day 2 Act 1 to set up model before training it
input_features = len(X_train_scaled[0])
hidden_layer_1 = 80
hidden_layer_2 = 30
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="linear", input_dim=input_features))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer_2, activation="sigmoid"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))
# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                1440      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 3901 (15.24 KB)
Trainable params: 3901 (15.24 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:

# Compile the model
#nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#https://stackoverflow.com/questions/33846069/how-to-set-rmse-cost-function-in-tensorflow
#mae=mean absolute error
rmse = tf.keras.metrics.RootMeanSquaredError()
nn.compile(optimizer="sgd", loss='mean_squared_error',
              metrics=[rmse,'mae'])



In [ ]:
#https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
#Save weights every 5 epochs
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="",
    #period=5,
    save_freq = 5
  )

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
10404/10404 [==============================] - 23s 2ms/step - loss: 6382170624.0000 - root_mean_squared_error: 79888.4922 - mae: 33767.8477
Epoch 2/10
10404/10404 [==============================] - 22s 2ms/step - loss: 6251313664.0000 - root_mean_squared_error: 79065.2500 - mae: 34219.5117
Epoch 3/10
10404/10404 [==============================] - 23s 2ms/step - loss: 6370265600.0000 - root_mean_squared_error: 79813.9453 - mae: 34727.0664
Epoch 4/10
10404/10404 [==============================] - 23s 2ms/step - loss: 6674386944.0000 - root_mean_squared_error: 81696.9219 - mae: 35910.7695
Epoch 5/10
10404/10404 [==============================] - 23s 2ms/step - loss: 6670695424.0000 - root_mean_squared_error: 81674.3281 - mae: 35915.1953
Epoch 6/10
10404/10404 [==============================] - 22s 2ms/step - loss: 6671324672.0000 - root_mean_squared_error: 81678.1797 - mae: 35912.8203
Epoch 7/10
10404/10404 [==============================] - 23s 2ms/step - loss: 6672988160.0000

In [ ]:
# Evaluate the model using the test data
model_loss, model_rmse, model_mae = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, RMSE: {model_rmse}, mae: {model_mae}")

3468/3468 - 5s - loss: 6505751552.0000 - root_mean_squared_error: 80658.2422 - mae: 34677.3320 - 5s/epoch - 2ms/step
Loss: 6505751552.0, RMSE: 80658.2421875, mae: 34677.33203125


In [ ]:
#https://www.tutorialspoint.com/keras/keras_regression_prediction_using_mpl.htm
prediction = nn.predict(X_test_scaled)
#prediction = prediction.flatten()
prediction

3468/3468 [==============================] - 7s 2ms/step


array([[56993.25 ],
       [56993.25 ],
       [14500.098],
       ...,
       [29286.426],
       [69900.61 ],
       [69900.61 ]], dtype=float32)

In [ ]:
prediction.flatten()

array([56993.25 , 56993.25 , 14500.098, ..., 29286.426, 69900.61 ,
       69900.61 ], dtype=float32)

In [ ]:
y_test_reshape = y_test.reshape(-1, 1)
y_test_reshape

array([[10929.13],
       [79180.63],
       [48595.04],
       ...,
       [15125.36],
       [35820.35],
       [71667.38]])

In [ ]:
#https://www.tensorflow.org/api_docs/python/tf/keras/metrics/R2Score
metric = tf.keras.metrics.R2Score()
metric.update_state(y_test_reshape, prediction)
result = metric.result()
print(result)
result.numpy()

tf.Tensor(0.05245191, shape=(), dtype=float32)


0.05245191

Optimization attempt 1

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(max_depth=12)
dt.fit(X_train, y_train)
predictions = dt.predict(X_test_scaled)
dt.score(X_test_scaled, y_test)

-0.2569800678770544

Optimization attempt 2

In [ ]:
#Try linear regression machine learning
#https://www.turing.com/kb/scikit-learn-cheatsheet-methods-for-classification-and-regression
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
predictions = lr.predict(X_test_scaled)
lr.score(X_test_scaled, y_test)

0.3436209143524821

In [ ]:
#Alternate optimizers
#https://www.tensorflow.org/api_docs/python/tf/keras/optimizers